<a href="https://colab.research.google.com/github/ricoen/google-ee-data-extract/blob/main/ee_data_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Earth Engine Data Extract

Author [@ricoen](https://github.com/ricoen)<br>

MIT Licensed

In [ ]:
#Import Earth Engine
import ee

In [ ]:
#Authenticate EE with your account
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=NJfUhshRhOtm1APfzevsgUVEiIWEGzW7PcghS9m6x_k&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6KKZgoSXfXZNoiVH-oGpnaZVpjRpmgNpF_oi_Q9udAUo4C0WyCXlI

Successfully saved authorization token.


In [ ]:
#Initialize EE
ee.Initialize()

In [ ]:
#Set the dataset from EE Image Collection
dataset = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')

In [ ]:
#Set the dataset taken time
taken = dataset.filterDate('2021-01-01', '2021-01-31')

In [ ]:
#Get the first raster for visualize the data
first_raster = ee.Image(taken.first())

In [ ]:
#Get info from the selected raster
img = first_raster

img.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90],
   'data_type': {'precision': 'float', 'type': 'PixelType'},
   'dimensions': [1440, 600],
   'id': 'Albedo_inst'},
  {'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90],
   'data_type': {'precision': 'float', 'type': 'PixelType'},
   'dimensions': [1440, 600],
   'id': 'AvgSurfT_inst'},
  {'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90],
   'data_type': {'precision': 'float', 'type': 'PixelType'},
   'dimensions': [1440, 600],
   'id': 'CanopInt_inst'},
  {'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90],
   'data_type': {'precision': 'float', 'type': 'PixelType'},
   'dimensions': [1440, 600],
   'id': 'ECanop_tavg'},
  {'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90],
   'data_type': {'precision': 'float', 'type': 'PixelType'},
   'dimensions': [1440, 600],
   'id': 'ESoil_tavg'},
  {'crs': 'EPSG:4326',
   'crs_transfo

In [ ]:
#Get the temperature value from T band
temperature = img.select('Tair_f_inst')

In [ ]:
#Import the folium library.
import folium
from folium import plugins

In [ ]:
#Set the basemap for map
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    )
}

In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[-7.50, 111.25], zoom_start=3, height=1000)

# Add custom basemaps
basemaps['Google Satellite'].add_to(my_map)

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Temperature Params
temp_params = {
  'min': 206,
  'max': 328,
  'palette': ['green', 'yellow', 'red']}

# Add the elevation model to the map object.
my_map.add_ee_layer(temperature.updateMask(temperature.gt(0)), temp_params, 'Air Temperature')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [ ]:
#Set the AOI (Area of Interest)
aoi = ee.Geometry.Polygon([
          [
            [
              112.62535572052002,
              -7.988390117845741
            ],
            [
              112.64822959899902,
              -7.988390117845741
            ],
            [
              112.64822959899902,
              -7.968202639589894
            ],
            [
              112.62535572052002,
              -7.968202639589894
            ],
            [
              112.62535572052002,
              -7.988390117845741
            ]
          ]
        ], None, False)

#poi = ee.Geometry.Point([112.644105, -7.9514187])

In [ ]:
#Reduce the raster
def setProperty(image):
    dict = image.reduceRegion(ee.Reducer.mean(), aoi)
    return image.set(dict)

In [ ]:
#Calculate the raster property
withMean = taken.map(setProperty)

In [ ]:
#Convert temperature unit (Kelvin) to Celcius 
t_avg = withMean.aggregate_array('Tair_f_inst').getInfo()
t_c = [x - 273 for x in t_avg]

#print(t_c)

In [ ]:
#Get the date
from datetime import datetime as dt

time = withMean.aggregate_array('system:time_start').getInfo()
#Convert from miliseconds to true time (local time with 24-H format)
date = [x // 1000 for x in time]
captured = [dt.utcfromtimestamp(i).strftime('%X-%x') for i in date]

#print(captured)

In [ ]:
#Convert the data to numpy array
import numpy as np

#Temperature data to numpy array (optional)
t_arr = np.asarray(t_c, dtype = np.float32)
#Date to numpy array (optional)
t_arRound = np.around(t_arr, decimals=1)
date_arr = np.asarray(captured)

In [ ]:
#Just look inside the array
t_arRound

array([21.4, 24.6, 23.3, 21.3, 20.5, 20.3, 19.7, 19.6, 20.6, 23.2, 24.3,
       20.7, 19.8, 19.6, 19. , 19. , 21.6, 25.8, 23.9, 22.1, 20.2, 20.1,
       19.8, 19.6, 21.7, 24.8, 22.2, 21.5, 20.4, 20.1, 19.6, 19.2, 22.2,
       27.5, 24.2, 20.9, 20. , 19.7, 19.4, 19.6, 21.8, 26.4, 24.2, 23.1,
       20.5, 19.9, 19.5, 19.2, 21.9, 25.3, 22.5, 21. , 20.2, 20.2, 19.8,
       19.9, 19.8, 21. , 22.9, 21.6, 20.4, 20. , 19.5, 19. , 21.2, 24.8,
       23.2, 21. , 20.1, 19.6, 19.3, 19.1, 22.1, 25.4, 25.1, 22.9, 20.4,
       20. , 19.6, 19.2, 21.9, 25.3, 25.5, 23.9, 21.1, 20.5, 20.1, 19.9,
       23.1, 26. , 24.3, 21.7, 20.5, 20.2, 20. , 19.9, 21. , 24.2, 25. ,
       23.1, 19.9, 19.7, 19.2, 19.3, 20.9, 24.5, 24. , 22.5, 19.9, 19.5,
       19.3, 19.1, 20.5, 24.1, 23.1, 21.4, 19.6, 19.4, 19. , 18.7, 20.6,
       24.4, 23.7, 21.1, 19.3, 19. , 18.8, 19.1, 20.2, 26.2, 24.1, 23. ,
       20. , 20. , 19.8, 19.6, 20.7, 23.4, 24.9, 22.1, 20.1, 19.7, 18.7,
       18.7, 20.4, 21.4, 22.5, 20.1, 19.1, 19.4, 19

In [ ]:
#Just look inside the array
date_arr

array(['00:00:00-01/01/21', '03:00:00-01/01/21', '06:00:00-01/01/21',
       '09:00:00-01/01/21', '12:00:00-01/01/21', '15:00:00-01/01/21',
       '18:00:00-01/01/21', '21:00:00-01/01/21', '00:00:00-01/02/21',
       '03:00:00-01/02/21', '06:00:00-01/02/21', '09:00:00-01/02/21',
       '12:00:00-01/02/21', '15:00:00-01/02/21', '18:00:00-01/02/21',
       '21:00:00-01/02/21', '00:00:00-01/03/21', '03:00:00-01/03/21',
       '06:00:00-01/03/21', '09:00:00-01/03/21', '12:00:00-01/03/21',
       '15:00:00-01/03/21', '18:00:00-01/03/21', '21:00:00-01/03/21',
       '00:00:00-01/04/21', '03:00:00-01/04/21', '06:00:00-01/04/21',
       '09:00:00-01/04/21', '12:00:00-01/04/21', '15:00:00-01/04/21',
       '18:00:00-01/04/21', '21:00:00-01/04/21', '00:00:00-01/05/21',
       '03:00:00-01/05/21', '06:00:00-01/05/21', '09:00:00-01/05/21',
       '12:00:00-01/05/21', '15:00:00-01/05/21', '18:00:00-01/05/21',
       '21:00:00-01/05/21', '00:00:00-01/06/21', '03:00:00-01/06/21',
       '06:00:00-01/

In [ ]:
#Visualize The Data
import pandas as pd

#Convert from numpy array to Pandas Series
dates = pd.Series(date_arr)
temperature = pd.Series(t_arRound)
#Concatnate the columns of Date and Temperature
df = pd.concat([dates, temperature], axis=1)
df.columns = ['dates', 'temperature']

#Visualize the data with Plotly
import plotly.express as px

#Define x and y values
fig = px.line(df, x='dates', y='temperature', 
              title='Air Temperature on Malang City - GLDAS-2.1 (NASA)')
#Show the figure
fig.show()